# Análise comparativa de modelos

In [251]:
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer

from sklearn.impute import IterativeImputer
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

from sklearn.model_selection import GridSearchCV, cross_validate
from sklearn.svm import SVR

## 1. Obtenção de dados

In [252]:
df = pd.read_csv("../data/raw/data.csv")
display(df.head())
display(df.describe())

df_dict = pd.read_csv("../data/external/dictionary.csv")
display(df_dict)

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


,survived,pclass,age,sibsp,parch,fare
count,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


,variavel,descricao,tipo,subtipo
0,survived,Sobreviventes ou não do passageiro,quantitativa,discreta
1,pclass,Número da classe do passageiro,qualitativa,ordinal
2,sex,Sexo do passageiro,qualitativa,nominal
3,age,Idade do passageiro,quantitativa,discreta
4,sibsp,Quantidade de irmãos/cônjuges a bordo,quantitativa,discreta
5,parch,Quantidade de pais / crianças a bordo,quantitativa,discreta
6,fare,Tarifa paga pelo passageiro,quantitativa,discreta
7,embarked,Sigla do porto de Embarque,qualitativa,nominal
8,class,Nome da classe do passageiro,qualitativa,ordinal
9,who,Classificação do passageiro,qualitativa,nominal


## 2.Tratamento dos dados

In [253]:
#Número de dados faltantes
df.isnull().sum()

survived         0
pclass           0
sex              0
age            177
sibsp            0
parch            0
fare             0
embarked         2
class            0
who              0
adult_male       0
deck           688
embark_town      2
alive            0
alone            0
dtype: int64

In [254]:
quantitative_variables = df_dict.query("tipo == 'quantitativa'").variavel.to_list()
qualitative_variables = df_dict.query("tipo == 'qualitativa'").variavel.to_list()

In [255]:
# Modelos, quantitativa
imputer = IterativeImputer(estimator=LinearRegression())
imputer.fit(df[quantitative_variables])
df[quantitative_variables] = imputer.transform(df[quantitative_variables])
display(df.describe())

,survived,pclass,age,sibsp,parch,fare
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,0.383838,2.308642,29.688724,0.523008,0.381594,32.204208
std,0.486592,0.836071,13.464558,1.102743,0.806057,49.693429
min,0.000000,1.000000,-4.520309,0.000000,0.000000,0.000000
25%,0.000000,2.000000,22.000000,0.000000,0.000000,7.910400
50%,0.000000,3.000000,29.514819,0.000000,0.000000,14.454200
75%,1.000000,3.000000,35.000000,1.000000,0.000000,31.000000
max,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [256]:
# Imputação, qualitativa
imputer = SimpleImputer(strategy='constant', fill_value="MISSING")
imputer.fit(df[qualitative_variables])
df[qualitative_variables] = imputer.transform(df[qualitative_variables])
display(df['deck'].describe())

count         891
unique          8
top       MISSING
freq          688
Name: deck, dtype: object

In [257]:
#Número atual de dados faltantes
df.isnull().sum()

survived       0
pclass         0
sex            0
age            0
sibsp          0
parch          0
fare           0
embarked       0
class          0
who            0
adult_male     0
deck           0
embark_town    0
alive          0
alone          0
dtype: int64

## 3. Limpeza de Dados

In [258]:
def tratar_outliers(df):
    colunas_numericas = df.select_dtypes(include=[np.number]).columns
    
    for coluna in colunas_numericas:
        Q1 = df[coluna].quantile(0.25)
        Q3 = df[coluna].quantile(0.75)
        IQR = Q3 - Q1
        
        df[coluna] = np.where((df[coluna] < (Q1 - 1.5 * IQR)) | (df[coluna] > (Q3 + 1.5 * IQR)), np.nan, df[coluna])
        df[coluna].fillna(df[coluna].median(), inplace=True)

    return df

df_cleaned = tratar_outliers(df)

print(df_cleaned.head())

   survived pclass     sex   age  sibsp  parch    fare embarked  class    who  \
0       0.0      3    male  22.0    1.0    0.0   7.250        S  Third    man   
1       1.0      1  female  38.0    1.0    0.0  13.000        C  First  woman   
2       1.0      3  female  26.0    0.0    0.0   7.925        S  Third  woman   
3       1.0      1  female  35.0    1.0    0.0  53.100        S  First  woman   
4       0.0      3    male  35.0    0.0    0.0   8.050        S  Third    man   

  adult_male     deck  embark_town alive  alone  
0       True  MISSING  Southampton    no  False  
1      False        C    Cherbourg   yes  False  
2      False  MISSING  Southampton   yes   True  
3      False        C  Southampton   yes  False  
4       True  MISSING  Southampton    no   True  


## 4. Preparação de dados

Normalização, codificação e o tratamento de dados discrepantes e/ou faltantes.

In [259]:
target_variable = 'survived'
quantitative_variables = (
    df_dict
    .query("tipo == 'quantitativa' and variavel != @target_variable")
    .variavel
    .to_list()
)
qualitative_variables = (
    df_dict
    .query("tipo == 'qualitativa' and variavel != @target_variable")
    .variavel
    .to_list()
)

In [269]:
X = df.drop(columns = [target_variable])
y = df[[target_variable]]
y = y.values.ravel() 

In [270]:
# Tratamento de dados discrepantes
quantitative_preprocess = Pipeline([
   ('missing', SimpleImputer(strategy='median')), # Tratamento de dados ausentes
   ('normalization', StandardScaler()), # Normalização
])

qualitative_preprocess = Pipeline([
   ('missing', SimpleImputer(strategy='most_frequent')), # Tratamento de dados ausentes
   ('encoding', OneHotEncoder()), # Transformação de dados 
])

preprocess = ColumnTransformer([
    ('quantitative', quantitative_preprocess, quantitative_variables),
    ('qualitative', qualitative_preprocess, qualitative_variables)
])

## 5. Seleção de modelos

In [271]:
models = {
    'LinearRegression': LinearRegression(),
    'KNeighborsRegressor': KNeighborsRegressor(),
    'SVR': SVR()
}

params = {
    'LinearRegression': {},
    'KNeighborsRegressor': {'n_neighbors': [3, 5, 7]},
    'SVR': {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf']}
}

In [275]:
results = []

for model_name, model in models.items():
    model_pipeline = Pipeline(steps=[
        ('preprocess', preprocess), 
        ('model', GridSearchCV(model, params[model_name], cv=5))
    ])
    
    scores = cross_validate(
    model_pipeline, X, y, 
    cv=5, scoring=metrics, n_jobs=-1
    )
    
    results.append({
        'model': model_name,
        'scores': scores
    })

In [274]:
for result in results:
    print(f"Modelo: {result['model']}")
    print(f"Resultados: {result['scores']}\n")

Modelo: LinearRegression
Resultados: {'fit_time': array([0.06717587, 0.10406137, 0.09862328, 0.06599307, 0.08395171]), 'score_time': array([0.01256967, 0.02609277, 0.02877831, 0.01375914, 0.01215386]), 'test_neg_mean_squared_error': array([-1.45570178e-31, -1.04850223e-30, -6.92348011e-31, -2.40702292e-31,
       -1.72061283e-31]), 'test_neg_mean_absolute_error': array([-2.93371782e-16, -8.20192852e-16, -6.78296370e-16, -3.59886902e-16,
       -3.60198762e-16]), 'test_r2': array([1., 1., 1., 1., 1.]), 'test_neg_mean_absolute_percentage_error': array([-0.89106145, -1.75561798, -1.72050562, -0.78651685, -0.98455056])}

Modelo: KNeighborsRegressor
Resultados: {'fit_time': array([0.14316511, 0.19903612, 0.19778275, 0.13370275, 0.15823483]), 'score_time': array([0.03234839, 0.03238249, 0.02411556, 0.01513052, 0.01537871]), 'test_neg_mean_squared_error': array([-0.03848541, -0.03682896, -0.02122347, -0.04681648, -0.01685393]), 'test_neg_mean_absolute_error': array([-0.0707635 , -0.05805243, 

In [ ]:
formatted_results = []